In [ ]:
AZURE_OAI_ENDPOINT = "
AZURE_OAI_KEY = "
AZURE_OAI_DEPLOYMENT = ""

GPT_ENDPOINT = ""
GPT_API_KEY = ""


In [ ]:
#gpt + dalle 합치기

import requests
from openai import AzureOpenAI
import os
from PIL import Image
import json

# 설정 정보 (환경변수 또는 직접 입력)
azure_oai_endpoint = AZURE_OAI_ENDPOINT
azure_oai_key = AZURE_OAI_KEY
azure_oai_deployment = AZURE_OAI_DEPLOYMENT

# GPT로부터 그림 프롬프트 받기
def requests_gpt(user_text):
    endpoint = GPT_ENDPOINT
    headers = {
        "Content-Type": "application/json",
        "api-key": GPT_API_KEY
    }
    payload = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_text.strip()
                    }            
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        return content_text
    else:
        return "GPT 요청 실패"

# Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    api_version="2024-02-01",
    api_key=AZURE_OAI_KEY,
    azure_endpoint=AZURE_OAI_ENDPOINT
)

# 사용자 입력
user_text = input("그려줬으면 하는 내용을 입력하세요 (예: '봄날 벚꽃이 흩날리는 공원'): ")
if user_text.lower() == 'quit':
    exit()

# GPT로부터 그림 프롬프트 생성
gpt_prompt = requests_gpt(user_text)
print("GPT가 생성한 이미지 설명:", gpt_prompt)

# 이미지 생성 요청
result = client.images.generate(
    model=AZURE_OAI_DEPLOYMENT,
    prompt=gpt_prompt,
    n=1
)

json_response = json.loads(result.model_dump_json())

# 이미지 저장 경로 설정
image_dir = os.path.join(os.curdir, 'images')
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

image_path = os.path.join(image_dir, 'generated_image.png')

# 이미지 다운로드 및 저장
image_url = json_response["data"][0]["url"]
generated_image = requests.get(image_url).content
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

# 이미지 보기
image = Image.open(image_path)
image.show()
